In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import os
# import csv
# import re
computer = 'o_desktop'
#computer = 'other'
if (computer == 'o_desktop'):
    os.environ['NUMEXPR_MAX_THREADS'] = '24'

# For future use.
# import threading
# import multiprocessing


6